权限管理

    借助django中间件实现网站的权限校验
    
Auth模块

    只要是跟用户相关的登录、注册、校验、修改密码、注销、验证用户是否登录

# auth 模块

## auth 方法

```python
from django.contrib import auth
# 自动查找auth_user表，自动给密码加密再对比。对比成功返回查找到的user对象，对比失败返回None
usr_obj = auth.authenticate(request,username=username,password=password)
# 注意，括号内必须同时传入用户名和密码

# 判断当前用户是否存在
if user_obj:
    # 保存用户状态
    auth.login(request, user_obj)  # 就是在操作session表
    # 只要执行了auth.login方法，就可以在任何地方通过request.user获取到当前登录的用户对象
    
    
# 判断用户是否登录
request.user # 有两种情况，1.已登录返回登录对象 2.未登录返回AnonymousUser匿名用户
request.user.is_authenticated()

```

登录之后才能访问的视图函数

```python
from django.contrib.auth.decorators import login_required

@login_required(login_url='/login/')  # 局部配置：用户没有登录跳转到login_url后面指定的网址
def home(request):
    pass


# 全局配置，没有登录跳转到指定页面

# 在settins.py中配置
LOGIN_URL = '/login/'
```

用户密码修改

```python
# 先校验两次密码是否一致
if new_password == confirm_password:
    # 校验老密码是否正确
    is_right = request.user.check_password(old_password)  # 自动加密对比密码
    if is_right:
        # 修改密码
        request.user.set_password(new_password)
        request.user.save()  # 真正操作数据库，保存密码
```

注销功能

```python
@login_required
def logout(request):
    auth.logout(request)  # 相当于找到session表中的当前user信息进行删除，刷新
    return redirect('/login/')
```

注册功能

```python
from django.contrib.auth.models import User
def register(request):
    if request.method == "POST":
        username = request.POST.get('username')
        password = request.POST.get('password')
        
        # 操作auth_user表写入数据
        # User.objects.create()方法不能使用，此方法写入的密码是铭文的，后面对比会出错
        User.objects.create_user(username=username, password=password)
    return render(request, 'register.html')
```

## auth 模块表扩展

扩展auth_user表


第一种方式：一对一关系来关联表，**不推荐使用**

第二种方式：面向对象的继承

```python
class UserInfo(AbstractUser):
    """
    如果继承了AbstractUser,那么在执行数据库迁移命令的时候auth_user表就不会在创建出来了。
    而UserInfo表中会出现auth_user所有的字段外加自己扩展的字段
    
    使用的前提：
        1.在继承之前没有执行过数据库迁移命令
            auth_user没有被创建，如果当前库已经创建那么就重新换一个库
        2.继承的类里面不要覆盖AbstracyUser里面的字段名
            表里面的字段都不要动，只扩展额外字段即可
        3.需要在配置文件中告诉django你要用UserInfo替代auth_user
            AUTH_USER_MODEL = 'app01.USerInfo'  '应用名.表面'
    """
    phone = models.BigIntegerField()
```